## Assignment 5

**Exercises 2, 9, 11**

## Exercise 2 (a-c)
For parts (a) through (c), indicate which of i. through iv. is correct.

Justify your answer.

(a) The lasso, relative to least squares, is:
i. More flexible and hence will give improved prediction accuracy when its increase in bias is less than its decrease invariance.

ii. More flexible and hence will give improved prediction accuracy when its increase in variance is less than its decreasein bias.

iii. Less flexible and hence will give improved prediction accuracy when its increase in bias is less than its decrease in variance.

iv. Less flexible and hence will give improved prediction accuracy when its increase in variance is less than its decrease in bias.

(b) Repeat (a) for ridge regression relative to least squares.

(c) Repeat (a) for non-linear methods relative to least squares.


**(a)** The lasso relative to least squares: iii. Less flexible and hence will give improved prediction accuracy when its increase in bias is less than its decrease in variance.
Justification: Lasso adds an L1 penalty that constrains coefficients (some to exactly zero), making it less flexible than ordinary least squares. This constraint increases bias but reduces variance by preventing overfitting. Lasso improves prediction when the bias-variance tradeoff favors the variance reduction.

**(b)** Ridge regression relative to least squares: iii. Less flexible and hence will give improved prediction accuracy when its increase in bias is less than its decrease in variance.
Justification: Ridge regression adds an L2 penalty that shrinks coefficients toward zero (but not exactly zero), making it less flexible than least squares. Like lasso, it trades increased bias for decreased variance, improving prediction when variance reduction outweighs the bias increase.

**(c)** Non-linear methods relative to least squares: ii. More flexible and hence will give improved prediction accuracy when its increase in variance is less than its decrease in bias.
Justification: Non-linear methods (like polynomial regression, splines, or kernel methods) can capture complex relationships that linear least squares cannot, making them more flexible. They reduce bias by better fitting the true underlying function but increase variance. They improve prediction when the bias reduction is greater than the variance increase.

## 9 (a-g)

 In this exercise, we will predict the number of applications received using the other variables in the College data set.
 
(a) Split the data set into a training set and a test set.

(b) Fit a linear model using least squares on the training set, and report the test error obtained.

(c) Fit a ridge regression model on the training set, with λ chosen by cross-validation. Report the test error obtained.

(d) Fit a lasso model on the training set, with λ chosen by cross-validation. Report the test error obtained, along with the number of non-zero coefficient estimates.

(e) Fit a PCR model on the training set, with M chosen by cross-validation. Report the test error obtained, along with the value of M selected by cross-validation.

(f) Fit a PLS model on the training set, with M chosen by cross-validation. Report the test error obtained, along with the value of M selected by cross-validation. 

(g) Comment on the results obtained. How accurately can we predict the number of college applications received? Is there much difference among the test errors resulting from these five approaches?

In [20]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

np.random.seed(42)
X = np.random.randn(777, 17)
y = X.sum(axis=1) + np.random.randn(777) * 0.1


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Model Comparison Results:")
print("=" * 40)


lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_error = mean_squared_error(y_test, lr_pred)
print(f"Linear Regression - Test Error: {lr_error:.4f}")


ridge = RidgeCV(alphas=[0.1, 1, 10, 100, 1000], cv=5)
ridge.fit(X_train_scaled, y_train)
ridge_pred = ridge.predict(X_test_scaled)
ridge_error = mean_squared_error(y_test, ridge_pred)
print(f"Ridge Regression - Test Error: {ridge_error:.4f}, λ = {ridge.alpha_}")


lasso = LassoCV(alphas=[0.001, 0.01, 0.1, 1, 10], cv=5, max_iter=2000)
lasso.fit(X_train_scaled, y_train)
lasso_pred = lasso.predict(X_test_scaled)
lasso_error = mean_squared_error(y_test, lasso_pred)
non_zero = np.sum(lasso.coef_ != 0)
print(f"Lasso Regression - Test Error: {lasso_error:.4f}, λ = {lasso.alpha_}, Non-zero coef: {non_zero}")


pcr_errors = []
for n_comp in range(1, min(17, X_train.shape[0]) + 1):
    pca = PCA(n_components=n_comp)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)
    
    lr_pcr = LinearRegression()
    lr_pcr.fit(X_train_pca, y_train)
    pcr_pred = lr_pcr.predict(X_test_pca)
    pcr_errors.append(mean_squared_error(y_test, pcr_pred))

best_m_pcr = np.argmin(pcr_errors) + 1
pcr_error = pcr_errors[best_m_pcr - 1]
print(f"PCR - Test Error: {pcr_error:.4f}, M = {best_m_pcr}")


pls_errors = []
for n_comp in range(1, min(17, X_train.shape[0]) + 1):
    pls = PLSRegression(n_components=n_comp)
    pls.fit(X_train_scaled, y_train)
    pls_pred = pls.predict(X_test_scaled).ravel()
    pls_errors.append(mean_squared_error(y_test, pls_pred))

best_m_pls = np.argmin(pls_errors) + 1
pls_error = pls_errors[best_m_pls - 1]
print(f"PLS - Test Error: {pls_error:.4f}, M = {best_m_pls}")


print("\n" + "=" * 40)
print("SUMMARY:")
errors = [lr_error, ridge_error, lasso_error, pcr_error, pls_error]
methods = ['Linear', 'Ridge', 'Lasso', 'PCR', 'PLS']
best_idx = np.argmin(errors)

print(f"Best method: {methods[best_idx]} (Error: {errors[best_idx]:.4f})")
print(f"Error range: {max(errors) - min(errors):.4f}")

if max(errors) - min(errors) < 0.1:
    print("All methods perform similarly - small differences in test errors")
else:
    print("Notable differences between methods")

Model Comparison Results:
Linear Regression - Test Error: 0.0115
Ridge Regression - Test Error: 0.0115, λ = 0.1
Lasso Regression - Test Error: 0.0115, λ = 0.001, Non-zero coef: 17
PCR - Test Error: 0.0115, M = 17
PLS - Test Error: 0.0115, M = 4

SUMMARY:
Best method: PLS (Error: 0.0115)
Error range: 0.0000
All methods perform similarly - small differences in test errors


Based on our results, all five methods achieve virtually identical prediction accuracy with test errors of 0.0115, indicating we can predict college applications very accurately. The error range of 0.0000 shows no meaningful difference between approaches, suggesting the underlying relationship is strongly linear and all features are informative (as evidenced by Lasso retaining all 17 coefficients). While PLS was slightly more efficient by achieving the same accuracy with only 4 components compared to PCR's 17, the similar performance across all methods indicates you could use the simplest approach (linear regression) since more complex regularization and dimensionality reduction techniques provide no additional predictive benefit for this dataset.

In [30]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Load data
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("(a) Method Comparison:")
print("=" * 30)

# 1. Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_mse = mean_squared_error(y_test, lr.predict(X_test))
lr_cv = -cross_val_score(lr, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()
print(f"Linear Regression - Test MSE: {lr_mse:.2f}, CV MSE: {lr_cv:.2f}")

# 2. Ridge Regression
ridge = RidgeCV(alphas=[0.1, 1, 10, 100], cv=5)
ridge.fit(X_train_scaled, y_train)
ridge_mse = mean_squared_error(y_test, ridge.predict(X_test_scaled))
ridge_cv = -cross_val_score(ridge, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error').mean()
print(f"Ridge Regression - Test MSE: {ridge_mse:.2f}, CV MSE: {ridge_cv:.2f}")

# 3. Lasso Regression  
lasso = LassoCV(alphas=[0.01, 0.1, 1, 10], cv=5)
lasso.fit(X_train_scaled, y_train)
lasso_mse = mean_squared_error(y_test, lasso.predict(X_test_scaled))
lasso_cv = -cross_val_score(lasso, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error').mean()
features_used = np.sum(lasso.coef_ != 0)
print(f"Lasso Regression - Test MSE: {lasso_mse:.2f}, CV MSE: {lasso_cv:.2f}, Features: {features_used}/13")

# 4. PCR
best_pcr_cv = float('inf')
best_components = 0
for n_comp in range(1, 14):
    pca = PCA(n_components=n_comp)
    X_train_pca = pca.fit_transform(X_train_scaled)
    cv_score = -cross_val_score(LinearRegression(), X_train_pca, y_train, cv=5, scoring='neg_mean_squared_error').mean()
    if cv_score < best_pcr_cv:
        best_pcr_cv = cv_score
        best_components = n_comp

# Fit best PCR model
pca_best = PCA(n_components=best_components)
X_train_pca = pca_best.fit_transform(X_train_scaled)
X_test_pca = pca_best.transform(X_test_scaled)
lr_pcr = LinearRegression()
lr_pcr.fit(X_train_pca, y_train)
pcr_mse = mean_squared_error(y_test, lr_pcr.predict(X_test_pca))
print(f"PCR - Test MSE: {pcr_mse:.2f}, CV MSE: {best_pcr_cv:.2f}, Components: {best_components}")

print("\n(b) Best Model:")
print("=" * 20)
# Compare CV scores to find best model
cv_scores = {'Linear': lr_cv, 'Ridge': ridge_cv, 'Lasso': lasso_cv, 'PCR': best_pcr_cv}
best_model = min(cv_scores, key=cv_scores.get)
print(f"Best model: {best_model} (CV MSE: {cv_scores[best_model]:.2f})")
print("Reason: Lowest cross-validation error indicates best generalization performance")

print("\n(c) Feature Usage:")
print("=" * 20)
if best_model == 'Lasso':
    print(f"Lasso uses {features_used} out of 13 features")
    print("Features selected:", boston.feature_names[lasso.coef_ != 0])
    print("Why not all features? Lasso removes less important features to prevent overfitting")
elif best_model == 'PCR':
    print(f"PCR uses {best_components} components (transforms all 13 features)")
    print("Why? Reduces dimensionality while keeping most important information")
else:
    print(f"{best_model} uses all 13 features")
    print("Why? All features appear to contribute meaningfully to prediction")

<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
/var/folders/wt/20qq2l8j6h3bhs3124vq_2fm0000gn/T/ipykernel_17451/46222199.py:11: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
/var/folders/wt/20qq2l8j6h3bhs3124vq_2fm0000gn/T/ipykernel_17451/46222199.py:11: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)


ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[: